This crawler was developed to retrieve residential energy consumption invoices from Centrais Elétricas de Santa Catarina (CELESC) in .pdfs using the selenium-requests library

In [12]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
from seleniumrequests import Chrome 
from selenium.webdriver import ChromeOptions

In [13]:
options = ChromeOptions()
filename = os.path.join(os.getcwd(), 'faturas')
os.makedirs(filename, exist_ok=True)
profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # Disable Chrome's PDF Viewer
           "download.default_directory": filename,
           "download.extensions_to_open": "",
           "browser.helperApps.neverAsk.saveToDisk": "",
}

options.add_experimental_option("prefs", profile) #add preferences to option variable
#options.add_argument("--headless") #Headless sets crawler in hidden mode, however it won't allow to perform the downloads
driver = Chrome(options=options)

#crawler must enter the directory before making a post requests
driver.get('https://agenciaweb.celesc.com.br/AgenciaWeb/consultarHistoricoPagto/listaHistoricoFaturas.jsp?d-1335161-p=1');

In [14]:
payload = {
    'param_url': '/agencia/',
    'sqUnidadeConsumidora': , #add the consumer unit
    'numeroMedidor': 'false',
    'tpDocumento': 'CPF',
    'numeroDocumentoCPF': , #add the CPF
    'autenticarSemDocumento': 'false',
    'tipoUsuario': 'clienteUnCons'
}
driver.request('POST','https://agenciaweb.celesc.com.br/AgenciaWeb/autenticar/autenticar.do', data=payload)

#two step login
payload = {'senha': ''} #add the password
driver.request('POST','https://agenciaweb.celesc.com.br/AgenciaWeb/autenticar/validarSenha.do', data=payload);

In [15]:
#consult the invoices' history
driver.get('https://agenciaweb.celesc.com.br/AgenciaWeb/consultarHistoricoPagto/consultarHistoricoPagto.do')

#url address ends with 'p=1'
req = driver.request('GET','https://agenciaweb.celesc.com.br/AgenciaWeb/consultarHistoricoPagto/listaHistoricoFaturas.jsp?d-1335161-p=1')

#request finds the last index for 'p'
req = BeautifulSoup(req.content, 'lxml')
last = int(re.findall(r'\d+', req.find('a',text='Última')['href'][-3:])[0])

#get the documents' addresses
urls = list()
for i in range(1, last+1):
    url = 'https://agenciaweb.celesc.com.br/AgenciaWeb/consultarHistoricoPagto/listaHistoricoFaturas.jsp?d-1335161-p='+str(i)
    req = driver.request('GET', url)
    req = BeautifulSoup(req.content, 'lxml')
    urls+=list(map(lambda x: 'https://agenciaweb.celesc.com.br' + x['href'], req.find_all('a', {"name": "teste"})))

In [16]:
#download files
for url in urls:
    driver.get(url)
    time.sleep(1) #wait to finish the download
    #rename file by its url
    os.rename(os.path.join(filename,'SegundaViaFatura.pdf'), os.path.join(filename, url[-7:].replace('/','_')+'.pdf'))
driver.close()